In [0]:
from pyspark.sql.functions import col,broadcast,month,rank,dense_rank,desc
from pyspark.sql.window import Window


class Insights:

    def __init__(self):
        pass
    def derivate(self,inputs):
        pass

class ScenarioOne(Insights):
    """
    Most sold pizza in Chicken Category(Large, Medium, Small) month wise

    """
    def derivate(self,inputs):

        pizzaDf = inputs.get("pizzaDf")
        pizzaTypeDf = inputs.get("pizzaTypeDf")
        orderDetailsDf = inputs.get("orderDetailsDf")
        ordersDf = inputs.get("ordersDf")

        pizzaTypeFilteredDf =  pizzaTypeDf.filter(col('category') == 'Chicken')

        # pizzaTypeFilteredDf.show()
        # Joining pizzaDf with ordersDetails using pizza_id

        joinDFOne = orderDetailsDf\
        .join(broadcast(pizzaDf),"pizza_id")\
        .join(pizzaTypeFilteredDf,"pizza_type_id")\
        .join(ordersDf,"order_id")
        
        #print("creating temp table")

        joinDFOne.createOrReplaceTempView("pizzaOrderDtls")

        scenarioOneDf = spark.sql("select name as Pizza_Name,pizza_id as pizza_size ,count(order_id) as total_sales,month(order_date) as order_mon from pizzaOrderDtls group by Pizza_Name,pizza_size,order_mon order by total_sales desc")


        #  Applying Rank function
        windowSpec = Window.partitionBy("order_mon","Pizza_Name").orderBy(desc("total_sales"),"pizza_size")

        rank_df =scenarioOneDf.withColumn("Rank",rank().over(windowSpec))


        """
        Same filtering using spark SQL

        scenarioOneDf= spark.sql("select a.name,a.pizza_id, a.total_sales, a.order_mon  from ( select name,pizza_id ,count(order_id) as total_sales,month(order_date) as order_mon from pizzaOrderDtls group by name,order_mon) a qualify rank() over(partition by a.order_mon order by a.total_sales,a.pizza_id desc ) =1 ") 

        return scenarioOneDf.show()
        """

        finalDf = rank_df.filter(col("Rank") == 1)

        print("Most sold pizza in Chicken Category month wise")

        return finalDf.select("Pizza_Name","pizza_size","total_sales","order_mon").show()
    

class ScenarioTwo(Insights):
    """
    Most sold pizza in every Category overall
    """
    def derivate(self,inputs):

        pizzaDf = inputs.get("pizzaDf")
        pizzaTypeDf = inputs.get("pizzaTypeDf")
        orderDetailsDf = inputs.get("orderDetailsDf")
        ordersDf = inputs.get("ordersDf")

        joinDFTwo = orderDetailsDf\
        .join(broadcast(pizzaDf),"pizza_id")\
        .join(pizzaTypeDf,"pizza_type_id")\
        .join(ordersDf,"order_id")

        joinDFTwo.createOrReplaceTempView("pizzaOrderDtls")

        scenarioTwoDf = spark.sql("select max(name) as Pizza_Name ,count(order_id) as Total_Sales,Category from pizzaOrderDtls group by category")

        return scenarioTwoDf.show()


        



        